<a href="https://colab.research.google.com/github/Arturro-98/LLM/blob/main/RAG/LangChain_Multiple_doc_Chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Multiple documents retrieval using Chromadb and OpenAI

Chroma DB is an open-source embedding database (also known as a vector store) that makes it easy to build LLM apps by storing and retrieving embeddings and their metadata, as well as documents and queries. It’s designed to provide efficient, scalable, and flexible ways to store and search embeddings. Chroma DB enhances the overall performance and scalability of LLM applications by providing a robust backend for storing and querying vectorized data.

<br>

**Langchain dataloaders:**


https://docs.kanaries.net/topics/LangChain/langchain-document-loader

https://medium.com/@varsha.rainer/document-loaders-in-langchain-7c2db9851123

In [1]:
!pip install spacy==3.7.4 weasel==0.3.4
!pip install typer==0.9.0

!pip -q install langchain openai tiktoken chromadb
!pip install -U langchain-openai

!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
  Using cached typer-0.9.0-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.9.4
    Uninstalling typer-0.9.4:
      Successfully uninstalled typer-0.9.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.9.0 which is incompatible.
ERROR: pip's dependency resolver does n

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader

#from langchain.chat_models import ChatOpenAI

from langchain_openai import OpenAIEmbeddings # To use embeddings from OpenAI
from langchain_openai import ChatOpenAI # To use LLM from OpenAI

In [3]:
import textwrap

# This function takes one argument of type dict and for result key, it will wrap the value, makes it easier to read. Written because answer is a long sting that was printed in one line.
def response_wrap(resp):

    for key, value in resp.items():
        if key == 'result':
            print(f"{key}:")
            print("\n".join(textwrap.wrap(value, width=80)))

        elif key == 'source_documents':
            print('\nSources:')
            for doc in value:
                if 'source' in doc.metadata:
                    print(doc.metadata['source'])
        else:
            print(f"{key}: {value}")
        print()

**1. Mounting to gdrive** to get stored PDF files and copy them into Colab workspace (speed up fetching data process)

**2. Loading Key (OpenAI) stored in a file in gdrive**

In [6]:
# Connecting to Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


**Loading Key (OpenAI) stored in a file in gdrive**

In [7]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab_Notebooks/LLM-RAG/')
from get_access import get_func
key_and_token = get_func() # Function returns the list with OpenAI key[0] & HuggingFace token[1]

In [8]:
#https://www.youtube.com/watch?v=3yPBVii7Ct0&t=344s

import os
key = key_and_token[0] # Returned OpenAI key string from a function
os.environ["OPENAI_API_KEY"] = key

In [7]:
pdf_folder_path = f'{root_dir}Colab_Notebooks/LLM-RAG//Data/' # PDF files on gdrive
os.listdir(pdf_folder_path)

['Neuroscience-Psychology-and-Conflict-Management-1710202873._print.pdf',
 'Psychology-of-Human-Relations-1695056929._print.pdf',
 'Fundamentals-of-Psychological-Disorders.pdf']

In [8]:
os.makedirs('/content/Data') # Making 'Data' folder in Colab workspace to copy all documents

In [9]:
''' Copy all data from Gdrive into created Data folder in Colab'''

import shutil

data_dir = '/content/Data/'

files = os.listdir(pdf_folder_path)
for file in files:
    shutil.copy(os.path.join(pdf_folder_path, file), data_dir) # Copying all files into Colab workspace to speed up fetching data process

In [10]:
!pip install pypdf # Required for .load()
from langchain_community.document_loaders import PyPDFDirectoryLoader

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.1 MB/s eta 0:00:00


**Loading multiple PDF documents - dataloader**


In [11]:
data_path = '/content/Data/'
loader = PyPDFDirectoryLoader(data_path)

#Can try UnstructuredPDFLoader from from langchain.document_loaders import UnstructuredPDFLoader

In [12]:
documents = loader.load()

In [13]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

len(texts)

3308

In [16]:
texts[13] # Chunk of a document

Document(page_content='purposes only; and\nproduce, reproduce, and Share Adapted Material for NonCommercial purposes only.B.\nExceptions and Limitations. For the avoidance of doubt, where Exceptions and Limitations2.\napply to Your use, this Public License does not apply, and You do not need to comply with its\nterms and conditions.\nTerm. The term of this Public License is specified in Section 6(a). 3.\nMedia and formats; technical modifications allowed. The Licensor authorizes You to4.\nexercise the Licensed Rights in all media and formats whether now known or hereafter\ncreated, and to make technical modifications necessary to do so. The Licensor waives and/or\nagrees not to assert any right or authority to forbid You from making technical\nmodifications necessary to exercise the Licensed Rights, including technical modifications\nnecessary to circumvent Effective Technological Measures. For purposes of this Public\nLicense, simply making modifications authorized by this Section 2(a

# Creating a Data Base

Store PDF texts as Vector Store in folder db

While LangChain provides the framework for building and deploying AI applications, Chroma DB provides the database for storing and retrieving the vector embeddings that these applications use.

In [17]:
# Embed and store the texts from PDFs
# Store the embeddings on disk in folder name from persist_directory
persist_directory = 'db'

# Initialize embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [18]:
# Loading persisted database from disk, and use it as normal.
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding)

# Retriever

In [19]:
retriever = vectordb.as_retriever()

In [20]:
docs = retriever.invoke("Tell me something about Interpersonal communication?")

In [21]:
len(docs) # By default 4 relevant documents are return

4

In [22]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2}) # Setting to retreive 2 top most relevant documents

In [23]:
retriever.search_type

'similarity'

In [24]:
retriever.search_kwargs # 2 (documents) as it was set above

{'k': 2}

# List of openAI models

Using the json module to print it in a more readable format

In [26]:
from openai import OpenAI
import json

client = OpenAI(
    api_key=os.getenv('OPENAI_API_KEY')
)

models = client.models.list()

# Extract the data from the SyncPage[Model] object
model_list = []
for model in models.data:
    model_dict = {
        'id': model.id,
        'created': model.created,
        'object': model.object,
        'owned_by': model.owned_by
    }
    model_list.append(model_dict)

# Pretty print the models list
print(json.dumps(model_list, indent=2))

[
  {
    "id": "dall-e-3",
    "created": 1698785189,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "gpt-4-1106-preview",
    "created": 1698957206,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "whisper-1",
    "created": 1677532384,
    "object": "model",
    "owned_by": "openai-internal"
  },
  {
    "id": "davinci-002",
    "created": 1692634301,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "gpt-4-turbo-preview",
    "created": 1706037777,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "gpt-4-0125-preview",
    "created": 1706037612,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "babbage-002",
    "created": 1692634615,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "dall-e-2",
    "created": 1698798177,
    "object": "model",
    "owned_by": "system"
  },
  {
    "id": "gpt-3.5-turbo-16k",
    "created": 1683758102,
    "object": "model",
    "owned_by": "o

# Chain

In [27]:
# Set up the turbo LLM
model = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-0125' # default gpt-3.5-turbo
)

In [28]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm = model, # default OpenAI()
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [29]:
# full example
query = "Tell me something about Interpersonal communication"
llm_response = qa_chain.invoke(query)
response_wrap(llm_response)

query: Tell me something about Interpersonal communication

result:
Interpersonal communication focuses on the exchange of messages between two
people in various settings like personal relationships, friendships, and work
collaborations. It involves skills associated with effective communication and
can help individuals achieve personal and professional goals. It occurs in our
daily interactions, such as conversations with significant others, friends, and
colleagues. Effective interpersonal communication is essential for building
relationships and successful interactions.


Sources:
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf



In [30]:
llm_response #(type dict) 'source_documents' is the top document and Document is the second top document

{'query': 'Tell me something about Interpersonal communication',
 'result': 'Interpersonal communication focuses on the exchange of messages between two people in various settings like personal relationships, friendships, and work collaborations. It involves skills associated with effective communication and can help individuals achieve personal and professional goals. It occurs in our daily interactions, such as conversations with significant others, friends, and colleagues. Effective interpersonal communication is essential for building relationships and successful interactions.',
 'source_documents': [Document(page_content='7.1 Elem ents of Int erpersonal C ommunication\nLearning Objec tives\nBy th e en d of this sec tion, y ou will be able t o:\n•Descr ibe th e diff erences bet ween th e sen der an d receiver of a\nmessa ge.\n•Descr ibe th e skills associat ed with eff ective int erpersonal sk ills.\n•Identif y several diff erent w ays to create bet ter int ercultural\ninteractions

In [31]:
qa_chain.retriever.search_type , qa_chain.retriever.vectorstore

('similarity',
 <langchain_community.vectorstores.chroma.Chroma at 0x7d46d06aad10>)

In [32]:
print(dir(qa_chain.combine_documents_chain.llm_chain.prompt))

['Config', 'InputType', 'OutputType', '__abstractmethods__', '__add__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__config__', '__custom_root_type__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__exclude_fields__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_validators__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__include_fields__', '__init__', '__init_subclass__', '__iter__', '__json_encoder__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__parameters__', '__post_root_validators__', '__pre_root_validators__', '__pretty__', '__private_attributes__', '__reduce__', '__reduce_ex__', '__repr__', '__repr_args__', '__repr_name__', '__repr_str__', '__rich_repr__', '__ror__', '__schema_cache__', '__setattr__', '__setstate__', '__signature__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__try_update_forward_refs__', '__validators__

In [33]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages)

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))]


# To see what was passed to the chain:
1. Template text/System prompt
2. {context}: two top documents
3. Question/query

In [34]:
template = qa_chain.combine_documents_chain.llm_chain.prompt.messages #list len 2
template = template[0].prompt.template
print(template)

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


 **Creating a db.zip file** that contains all the files and folders within the db directory in the current directory

In [35]:
!zip -r db.zip ./db

  adding: db/ (stored 0%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/ (stored 0%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/length.bin (deflated 94%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/index_metadata.pickle (deflated 42%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/header.bin (deflated 56%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/link_lists.bin (deflated 83%)
  adding: db/b94c06c4-0739-4a91-bd66-e38b87641586/data_level0.bin (deflated 17%)
  adding: db/chroma.sqlite3 (deflated 38%)


# Deleting db folder/collection from colab

In [36]:
# Removing files and its directory (db/)
!rm -rf db/

# To start again from zip db:



1.   Restart runtime in Colab
2.   Get openAI key and make all imports from beginning of the notebook. Plus execute response_wrap() function



In [4]:
!unzip db.zip

Archive:  db.zip
   creating: db/
   creating: db/b94c06c4-0739-4a91-bd66-e38b87641586/
  inflating: db/b94c06c4-0739-4a91-bd66-e38b87641586/length.bin  
  inflating: db/b94c06c4-0739-4a91-bd66-e38b87641586/index_metadata.pickle  
  inflating: db/b94c06c4-0739-4a91-bd66-e38b87641586/header.bin  
  inflating: db/b94c06c4-0739-4a91-bd66-e38b87641586/link_lists.bin  
  inflating: db/b94c06c4-0739-4a91-bd66-e38b87641586/data_level0.bin  
  inflating: db/chroma.sqlite3       


In [9]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb2 = Chroma(persist_directory=persist_directory,
                  embedding_function=embedding,
                   )

retriever = vectordb2.as_retriever(search_kwargs={"k": 2})

In [10]:
# Set up the turbo LLM
model = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo-0125' # default gpt-3.5-turbo
)

In [11]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=model,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [12]:
# full example
query = "Tell me something about Interpersonal communication"
llm_response = qa_chain.invoke(query)
response_wrap(llm_response)

query: Tell me something about Interpersonal communication

result:
Interpersonal communication focuses on the exchange of messages between two
people in various settings like at home, with friends, or at work. It involves
interactions such as saying good morning to a significant other, discussing life
events with a friend, or collaborating with a coworker on a project. Effective
interpersonal communication skills are essential for achieving personal and
professional goals. It involves understanding the differences between the sender
and receiver of a message, developing effective interpersonal skills, and
creating better interactions, including intercultural interactions.


Sources:
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf



In [13]:
query = "What is genesis of Abnormal Behavior?"
llm_response = qa_chain.invoke(query)
response_wrap(llm_response)

query: What is genesis of Abnormal Behavior?

result:
The genesis of abnormal behavior can be attributed to a combination of factors
such as personal distress, psychological dysfunction, deviance from social
norms, dangerousness to self and others, and costliness to society. Abnormal
behavior is not just a result of one specific cause but rather a complex
interplay of various factors that contribute to maladaptive behavior.


Sources:
/content/Data/Fundamentals-of-Psychological-Disorders.pdf
/content/Data/Fundamentals-of-Psychological-Disorders.pdf



In [14]:
query = "Powiedz mi cos o emocjach w psychologii"
llm_response = qa_chain.invoke(query)
response_wrap(llm_response)

query: Powiedz mi cos o emocjach w psychologii

result:
Emocje w psychologii są złożonymi reakcjami organizmu na bodźce zewnętrzne lub
wewnętrzne. Badania nad emocjami obejmują różnorodne aspekty, takie jak
wyrażanie emocji, rozpoznawanie emocji u innych, wpływ emocji na zachowanie i
zdrowie psychiczne. Emocje odgrywają istotną rolę w relacjach międzyludzkich i w
procesie podejmowania decyzji.


Sources:
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf
/content/Data/Psychology-of-Human-Relations-1695056929._print.pdf



In [15]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template) # System prompt

Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
{context}


In [16]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

{question}
